In [ ]:
# REST API Backend em Python - Estrutura Básica

5

In [ ]:
# 1. Instalação de dependências
# Execute no terminal:
# pip install fastapi uvicorn pydantic sqlalchemy python-dotenv

## Estrutura do Projeto

```
projeto/
├── main.py                 # Arquivo principal da aplicação
├── config.py              # Configurações
├── models.py              # Modelos de dados
├── schemas.py             # Schemas Pydantic
├── routes/
│   ├── __init__.py
│   ├── users.py           # Endpoints de usuários
│   └── items.py           # Endpoints de itens
├── database.py            # Configuração do banco
├── requirements.txt       # Dependências
└── .env                   # Variáveis de ambiente
```

In [ ]:
# 2. MAIN.PY - Arquivo Principal
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

app = FastAPI(
    title="Minha API",
    description="REST API Backend",
    version="1.0.0"
)

# Configurar CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Importar rotas
# from routes import users, items
# app.include_router(users.router)
# app.include_router(items.router)

# Rota básica
@app.get("/")
async def root():
    return {"message": "Bem-vindo à API"}

@app.get("/health")
async def health():
    return {"status": "OK"}

if __name__ == "__main__":
    # Executar: python main.py
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
# 3. SCHEMAS.PY - Modelos Pydantic (Validação de dados)
from pydantic import BaseModel
from typing import Optional
from datetime import datetime

class UserBase(BaseModel):
    name: str
    email: str

class UserCreate(UserBase):
    password: str

class User(UserBase):
    id: int
    created_at: datetime
    
    class Config:
        from_attributes = True

class ItemBase(BaseModel):
    title: str
    description: Optional[str] = None
    price: float

class ItemCreate(ItemBase):
    pass

class Item(ItemBase):
    id: int
    owner_id: int
    created_at: datetime
    
    class Config:
        from_attributes = True

In [ ]:
# 4. MODELS.PY - Modelos SQLAlchemy (Banco de dados)
from sqlalchemy import Column, Integer, String, Float, DateTime, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from datetime import datetime

Base = declarative_base()

class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    email = Column(String, unique=True, index=True)
    password = Column(String)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    items = relationship("Item", back_populates="owner")

class Item(Base):
    __tablename__ = "items"
    
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String, index=True)
    description = Column(String)
    price = Column(Float)
    owner_id = Column(Integer, ForeignKey("users.id"))
    created_at = Column(DateTime, default=datetime.utcnow)
    
    owner = relationship("User", back_populates="items")

In [ ]:
# 5. DATABASE.PY - Configuração de conexão
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os
from dotenv import load_dotenv

load_dotenv()

# Usar SQLite para desenvolvimento (trocar por PostgreSQL em produção)
DATABASE_URL = os.getenv("DATABASE_URL", "sqlite:///./test.db")

engine = create_engine(
    DATABASE_URL, 
    connect_args={"check_same_thread": False} if "sqlite" in DATABASE_URL else {}
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

In [ ]:
# 6. ROUTES/USERS.PY - Endpoints de Usuários
from fastapi import APIRouter, Depends, HTTPException
from sqlalchemy.orm import Session
# from database import get_db
# from models import User as UserModel
# import schemas

router = APIRouter(prefix="/users", tags=["users"])

@router.get("/")
async def list_users(db: Session = Depends(get_db)):
    """Listar todos os usuários"""
    # users = db.query(UserModel).all()
    return {"users": []}

@router.post("/")
async def create_user(user: UserCreate, db: Session = Depends(get_db)):
    """Criar novo usuário"""
    # db_user = UserModel(**user.dict())
    # db.add(db_user)
    # db.commit()
    # db.refresh(db_user)
    # return db_user
    return {"message": "Usuário criado"}

@router.get("/{user_id}")
async def get_user(user_id: int, db: Session = Depends(get_db)):
    """Obter usuário por ID"""
    # user = db.query(UserModel).filter(UserModel.id == user_id).first()
    # if not user:
    #     raise HTTPException(status_code=404, detail="Usuário não encontrado")
    # return user
    return {"user_id": user_id}

In [ ]:
# 7. REQUIREMENTS.TXT - Dependências do projeto
"""
fastapi==0.104.1
uvicorn==0.24.0
sqlalchemy==2.0.23
pydantic==2.5.0
python-dotenv==1.0.0
pydantic-settings==2.1.0

# Opcional para produção:
# psycopg2-binary==2.9.9  (PostgreSQL)
# gunicorn==21.2.0        (Servidor WSGI)
# alembic==1.12.1         (Migrations)
# python-jose==3.3.0      (JWT Auth)
# passlib==1.7.4          (Senha criptografada)
"""

## Como Executar

### 1. Criar ambiente virtual
```bash
python -m venv venv
venv\Scripts\activate  # Windows
```

### 2. Instalar dependências
```bash
pip install -r requirements.txt
```

### 3. Executar a API
```bash
python main.py
# ou
uvicorn main:app --reload
```

### 4. Acessar a documentação interativa
- Swagger UI: http://localhost:8000/docs
- ReDoc: http://localhost:8000/redoc

## Estrutura de Endpoints Básicos

| Método | Endpoint | Descrição |
|--------|----------|-----------|
| GET | / | Raiz da API |
| GET | /health | Verificar saúde da API |
| GET | /users | Listar usuários |
| POST | /users | Criar usuário |
| GET | /users/{id} | Obter usuário |